# Handwritten Digit Recognition with Neural Network (from Scratch)
This notebook builds a simple neural network using only NumPy, Pandas to classify digits from the MNIST dataset.
We’ll walk through data loading, preprocessing, model creation, training, and evaluation.

## 1. Importing Required Libraries
We start by importing NumPy for numerical operations, pandas for data handling, and matplotlib for visualization.


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Loaded dataset with shape: (42000, 785)


## 2. Loading the Dataset
We load the `train.csv` file which contains labeled images of handwritten digits. Each row represents one image.

In [ ]:
# Load the MNIST training dataset
data = pd.read_csv("train.csv")

# Display the shape of the dataset
print(f"Loaded dataset with shape: {data.shape}")

## 3. Exploring the Dataset
We convert the dataset to a NumPy array and print its dimensions to understand how many examples and features we have.

In [13]:
# Convert DataFrame to NumPy array and get dataset shape
data = np.array(data)
m, n = data.shape
print(f"Dataset contains {m} examples with {n} features (including the label).")

Dataset contains 42000 examples with 785 features (including the label).


## 4. Data Preprocessing
We shuffle the data, normalize pixel values, and split it into training and development sets.
Normalization helps the model train faster by scaling pixel values to the [0,1] range.

In [ ]:
# Shuffle the dataset to avoid any ordering bias
np.random.shuffle(data)

# Split the dataset into training and dev sets
# Transpose so that each column represents one example
data_dev = data[1:1000].T
X_dev = data_dev[1:] / 255.0  # Normalize pixel values
Y_dev = data_dev[0]

data_train = data[1000:].T
X_train = data_train[1:] / 255.0  # Normalize pixel values
Y_train = data_train[0]

# Get the number of training examples
_, m_train = X_train.shape

## 5. Neural Network Implementation

### 5.1 Neural Network Architecture Overview
We build a simple feedforward neural network with the following structure:
- **Input layer**: 784 units (one for each pixel in a 28×28 grayscale image)
- **Hidden layer**: 10 neurons with ReLU activation
- **Output layer**: 10 neurons with softmax activation, one for each digit class (0–9)

### 5.2 Initializing Parameters
We randomly initialize the weights and biases for both layers.
This helps break symmetry and allows the network to begin learning.

### 5.3 Activation Functions
We define the activation functions used in the network:
- **ReLU (Rectified Linear Unit)** for the hidden layer
- **Softmax** for the output layer to convert scores into probabilities

### 5.4 Forward Propagation
We calculate the intermediate activations and output probabilities by applying the forward pass of the network.

### 5.5 One-Hot Encoding
Since our labels are digits (0–9), we convert them into one-hot encoded vectors to compute the loss and gradients properly.

### 5.6 Backward Propagation
We compute gradients of the loss with respect to all parameters using the chain rule.
This allows us to update the weights and biases to minimize the prediction error.

### 5.7 Updating Parameters
Using the gradients computed during backpropagation, we update the weights and biases via gradient descent.

### 5.8 Making Predictions and Evaluating Accuracy
We define functions to:
- Make predictions using the trained model
- Evaluate how well the model performs by calculating accuracy on the training and dev sets

### 5.9 Training the Model
We run gradient descent over multiple iterations to train the model.
During training, we monitor the accuracy to observe learning progress.


In [ ]:
class NeuralNetwork:
    def __init__(self, input_size=784, hidden_size=10, output_size=10):
        self.W1 = np.random.rand(hidden_size, input_size) - 0.5
        self.b1 = np.random.rand(hidden_size, 1) - 0.5
        self.W2 = np.random.rand(output_size, hidden_size) - 0.5
        self.b2 = np.random.rand(output_size, 1) - 0.5

    def ReLU(self, Z):
        return np.maximum(0, Z)

    def ReLU_deriv(self, Z):
        return Z > 0

    def softmax(self, Z):
        A= np.exp(Z)/sum(np.exp(Z))
        return A

    def forward(self, X):
        Z1 = self.W1 @ X + self.b1
        A1 = self.ReLU(Z1)
        Z2 = self.W2 @ A1 + self.b2
        A2 = self.softmax(Z2)
        return Z1, A1, Z2, A2

    def one_hot(self, Y):
        one_hot_Y = np.zeros((Y.max()+1, Y.size))
        one_hot_Y[Y, np.arange(Y.size)] = 1
        return one_hot_Y

    def backward(self, X, Y, Z1, A1, A2):
        m = X.shape[1]
        one_hot_Y = self.one_hot(Y)
        dZ2 = A2 - one_hot_Y
        dW2 = 1/m * dZ2 @ A1.T
        db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)
        dZ1 = (self.W2.T @ dZ2) * self.ReLU_deriv(Z1)
        dW1 = 1/m * dZ1 @ X.T
        db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)
        return dW1, db1, dW2, db2

    def update(self, dW1, db1, dW2, db2, alpha):
        self.W1 -= alpha * dW1
        self.b1 -= alpha * db1
        self.W2 -= alpha * dW2
        self.b2 -= alpha * db2

    def train(self, X, Y, iterations, alpha):
        for i in range(iterations):
            Z1, A1, Z2, A2 = self.forward(X)
            dW1, db1, dW2, db2 = self.backward(X, Y, Z1, A1, A2)
            self.update(dW1, db1, dW2, db2, alpha)
            if i % 10 == 0:
                preds = self.predict(X)
                acc = self.accuracy(preds, Y)
                print(f"Iteration {i} - Accuracy: {acc:.4f}")

    def predict(self, X):
        _, _, _, A2 = self.forward(X)
        return np.argmax(A2, axis=0)

    def accuracy(self, preds, Y):
        return np.mean(preds == Y)


In [ ]:
nn = NeuralNetwork()
nn.train(X_train, Y_train, iterations=500, alpha=0.1)
